In [1]:
import pickle
import pandas as pd

In [2]:
with open('./data/transformed_data/teams_stats.pickle', 'rb') as team_stats:
    teams_stats = pickle.load(team_stats)

In [3]:
with open('./data/transformed_data/players_stats.pickle', 'rb') as play_stats:
    players_stats = pickle.load(play_stats)

In [4]:
with open('./data/transformed_data/players_advanced_stats.pickle', 'rb') as play_advanced_stats:
    players_advanced_stats = pickle.load(play_advanced_stats)

In [5]:
with open(f'./data/transformed_data/playoffs.pickle', 'rb') as playoff:
    playoffs = pickle.load(playoff)

In [6]:
game_types = ["playoffs", "leagues"]
years = list(range(2001, 2024))
stats = {}
df_columns = ["ID", "age", "games_played_perc", "games_started_perc", "avg_minutes_played", "WS48", "team_successes", 
              "defensive", "most_improved", "most_valuable", "most_valuable_finals", "sixth_man", "all_league_1", "all_league_2", 
              "all_league_3", "all_league_4", "all_league_5", "all_league_6", "all_league_7", "all_league_8", "all_league_9", "all_league_10", 
              "all_league_11", "all_league_12", "all_league_13", "all_league_14", "all_league_15", "all_def_1", "all_def_2", "all_def_3", 
              "all_def_4", "all_def_5", "all_def_6", "all_def_7", "all_def_8", "all_def_9", "all_def_10", "all_def_11"]

In [7]:
# % GAMES PLAYED FOR EACH PLAYER
for year in years:  
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    stats[year] = pd.DataFrame(columns=df_columns)
    stats[year]['ID'] = [player for player in player_list]
    stats[year] = stats[year].set_index("ID")
    for player_id in player_list:
        games_played = 0
        team_games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    player_team = players_stats[year][game_type].query(f"ID == '{player_id}'")["Tm"].iloc[0]
                except:
                    pass
                try:
                    games_played += players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]   
                except:
                    pass
                try:
                    team_games_played += teams_stats[year][game_type].query(f"Team == '{player_team}'")["G"].iloc[0]
                except:    
                    pass

        perc_games_played = games_played / team_games_played
            
        stats[year].loc[[f'{player_id}'], ['games_played_perc']] = perc_games_played

In [8]:
# % GAMES STARTED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        games_started = 0
        team_games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    player_team = players_stats[year][game_type].query(f"ID == '{player_id}'")["Tm"].iloc[0]
                except:
                    pass
                try:
                    games_started += players_stats[year][game_type].query(f"ID == '{player_id}'")["GS"].iloc[0]
                except:
                    pass
                try:    
                    team_games_played += teams_stats[year][game_type].query(f"Team == '{player_team}'")["G"].iloc[0]
                except:
                    pass

        perc_games_started = games_started / team_games_played
            
        stats[year].loc[[f'{player_id}'], ['games_started_perc']] = perc_games_started

In [9]:
# AVERAGE MINUTES PLAYED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        avg_minutes_played = 0
        avg_minutes_played_x_games = 0
        total_games = 0
        minutes_played_per_game = 0
        games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played_per_game = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    games_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]
                except:
                    minutes_played_per_game += 0
                    games_played += 0
                
            avg_minutes_played_x_games += minutes_played_per_game * games_played
            total_games += games_played
            
        if total_games != 0:
            avg_minutes_played = round(avg_minutes_played_x_games/total_games, 2)
        else:
            avg_minutes_played = 0
        
        stats[year].loc[[f'{player_id}'], ['avg_minutes_played']] = avg_minutes_played

In [10]:
# WIN SHARES PER 48 MINUTES FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        minutes_played = 0
        win_shares = 0
        win_share_48 = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    win_shares = players_advanced_stats[year][game_type].query(f"ID == '{player_id}'")["WS"].iloc[0]
                except:
                    minutes_played += 0
                    win_shares += 0
            
        if minutes_played != 0:
            win_share_48 = round((win_shares/minutes_played) * 48, 2)
        else:
            win_share_48 = 0
        
        stats[year].loc[[f'{player_id}'], ['WS48']] = win_share_48

In [11]:
# AGE FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        try:
            stats[year].loc[[f'{player_id}'], ['age']] = players_stats[year]["leagues"].query(f"ID == '{player_id}'")["Age"].iloc[0]
        except:
            stats[year].loc[[f'{player_id}'], ['age']] = players_stats[year]["playoffs"].query(f"ID == '{player_id}'")["Age"].iloc[0]

In [13]:
personal_awards = pd.read_csv('./data/transformed_data/personal_awards.csv')

In [14]:
awards_columns = ["defensive", "most_improved", "most_valuable", "most_valuable_finals", "sixth_man", "all_league_1", "all_league_2", 
                "all_league_3", "all_league_4", "all_league_5", "all_league_6", "all_league_7", "all_league_8", "all_league_9", "all_league_10", 
                "all_league_11", "all_league_12", "all_league_13", "all_league_14", "all_league_15", "all_def_1", "all_def_2", "all_def_3", 
                "all_def_4", "all_def_5", "all_def_6", "all_def_7", "all_def_8", "all_def_9", "all_def_10", "all_def_11"]

In [15]:
# PERSONAL AWARDS FOR EACH PLAYER
for year in years:
    stats[year] = stats[year].fillna(0)
    for column in awards_columns:
        player_id = personal_awards.query(f"year == {year}")[f"{column}"].iloc[0]
        if not pd.isna(player_id):
            stats[year].loc[player_id, [f"{column}"]] = 1

In [32]:
distinct_teams = pd.read_csv('./data/transformed_data/distinct_teams.csv')

In [36]:
playoffs_scores = {}
for year in years:    
    playoffs_scores[year] = {}
    playoffs_scores[year] = distinct_teams.copy()
    playoffs_scores[year]['score'] = 0
    for index, row in playoffs_scores[year].iterrows():
        team = row["ID"]
        check_team_history = playoffs[year][playoffs[year].isin([f'{team}'])].stack()
        if check_team_history.empty:
            pass
        else:
            stage_id = check_team_history.index[0][0]
            if stage_id == 0:
                stage_outcome = check_team_history.index[0][1]
                if stage_outcome == "winner":
                    playoffs_scores[year].loc[index, 'score'] = 5
                else: 
                    playoffs_scores[year].loc[index, 'score'] = 3
            elif stage_id in [1, 2]:
                playoffs_scores[year].loc[index, 'score'] = 2
            elif stage_id in list(range(3, 15)):
                playoffs_scores[year].loc[index, 'score'] = 1

In [41]:
stats[2001]

,age,games_played_perc,games_started_perc,avg_minutes_played,WS48,team_successes,defensive,most_improved,most_valuable,most_valuable_finals,...,all_def_2,all_def_3,all_def_4,all_def_5,all_def_6,all_def_7,all_def_8,all_def_9,all_def_10,all_def_11
ID,,,,,,,,,,,,,,,,,,,,,
armstda01,32,0.918605,0.918605,37.15,11.32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
madsema01,25,0.846939,0.030612,8.34,5.74,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doolike01,20,0.775510,0.010204,16.30,4.12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
piercpa01,23,1.000000,1.000000,38.00,13.14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mckeyde01,34,0.813953,0.232558,14.65,4.48,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
portech01,22,0.510000,0.350000,22.50,0.43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cheanca01,29,0.109756,0.060976,17.00,-0.28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
potapvi01,25,1.000000,0.085366,23.20,7.03,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
playoffs_scores[2001]

,ID,team,score
0,ATL,Atlanta Hawks,0
1,BOS,Boston Celtics,0
2,BRK,Brooklyn Nets,0
3,CHI,Chicago Bulls,0
4,CHO,Charlotte Hornets,1
5,CLE,Cleveland Cavaliers,0
6,DAL,Dallas Mavericks,1
7,DEN,Denver Nuggets,0
8,DET,Detroit Pistons,0
9,GSW,Golden State Warriors,0


In [43]:
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        try:
            player_team = players_stats[year]["leagues"].query(f"ID == '{player_id}'")["Tm"].iloc[0]
        except:
            player_team = players_stats[year]["playoffs"].query(f"ID == '{player_id}'")["Tm"].iloc[0]
        
        stats[year].loc[[f'{player_id}'], ['team_successes']] = playoffs_scores[year].query(f"ID == '{player_team}'")["score"].iloc[0]

In [44]:
stats[2001]

,age,games_played_perc,games_started_perc,avg_minutes_played,WS48,team_successes,defensive,most_improved,most_valuable,most_valuable_finals,...,all_def_2,all_def_3,all_def_4,all_def_5,all_def_6,all_def_7,all_def_8,all_def_9,all_def_10,all_def_11
ID,,,,,,,,,,,,,,,,,,,,,
armstda01,32,0.918605,0.918605,37.15,11.32,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
madsema01,25,0.846939,0.030612,8.34,5.74,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doolike01,20,0.775510,0.010204,16.30,4.12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
piercpa01,23,1.000000,1.000000,38.00,13.14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mckeyde01,34,0.813953,0.232558,14.65,4.48,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
portech01,22,0.510000,0.350000,22.50,0.43,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cheanca01,29,0.109756,0.060976,17.00,-0.28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
potapvi01,25,1.000000,0.085366,23.20,7.03,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
players_stats[2001]["leagues"].query("ID == 'armstda01'")

,Player,Age,Tm,G,GS,MP,ID
15,Darrell Armstrong,32,ORL,75,75,36.9,armstda01
